In [7]:
import os
import requests
import json
from json import JSONEncoder
import pandas as pd
from datetime import datetime,timezone, timedelta
from tqdm import tqdm
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import base64
from urllib.parse import urlencode
import calendar
import pytz
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import xlsxwriter
from tzlocal import get_localzone
import dateutil
import pandas_gbq
from google.oauth2 import service_account
import config

In [20]:
pd.options.mode.chained_assignment = None

In [70]:
class Spotify_UserInput:
    user_time_range = {"1":"short_term", "2":"medium_term", "3":"long_term"}

    time_range_reminder = ["1. short_term = approximately last 4 weeks", "2. medium_term = approximately last 6 months",
                          "3. long_term = calculated from several years of data and including all new data as it becomes available"]
    
    def user_timerange(user_time_range, time_range_reminder):
        '''
        Ask user for specify time range
        - short_term = approximately last 4 weeks
        - medium_term = approximately last 6 months
        - long_term = calculated from several years of data and including all new data as it becomes available
        '''
        while True:
            print("How far back would you like to go?"
                  "\nSelect time range by typing the number 1-3, your choices: "
                  "\n1. short_term = approximately last 4 weeks"
                  "\n2. medium_term = approximately last 6 months"
                  "\n3. long_term = calculated from several years of data and including all new data as it becomes available"
                  "\nYour input: ")
            date_range = input("Time range #: ")
            i = 0

            while date_range not in user_time_range.keys():
                date_range = input("Invalid entry. Please select time range with number above: ")
                i += 1
                if i == 2:
                    print("\nLooks like you're having trouble. Here are the available inputs: \n")
                    print(time_range_reminder)
                    print()
                    date_range = input("Enter a court number: ")
                    i += 1
                if i >= 3 and i % 3:
                    quitting = input("Do you want to quit (yes/no)? ").lower()
                    if quitting == "yes":
                        break
                    else:
                        date_range = input("Select time range by typing the number 1-3, your choices: "
                                  "\n1. short_term = approximately last 4 weeks"
                                  "\n2. medium_term = approximately last 6 months"
                                  "\n3. long_term = calculated from several years of data and including all new data as it becomes available"
                                  "\nYour input: ")
                if court not in COURT_DATA_SEARCH.keys():
                    quitting = "yes"
                    return quitting
            print(f"Pulling '{user_time_range[date_range]}' data...")
            return user_time_range[date_range]

    
class Spotify_Auth:
    client_id = config.client_id
    client_secret = config.client_secret
    username = config.username
    
    def auth(client_id, client_secret, username):
        redirect_uri = 'http://localhost:7777/callback'
        scope = 'user-read-recently-played, user-top-read'

        auth_token = util.prompt_for_user_token(username=username, 
                                          scope=scope, 
                                          client_id=client_id,   
                                          client_secret=client_secret,     
                                          redirect_uri=redirect_uri)
        return auth_token

    
class Spotify_Data:
    
    def recently_played(auth_token):
        base_url = 'https://api.spotify.com/v1/me/player/recently-played?'
        #track_id = '6y0igZArWVi6Iz0rj35c1Y'

        #2. Authentication
        #3. Parameters -- would be stored with authentication
        headers = {
            "Authorization": f"Bearer {auth_token}"
        }

        #4. Create an empty list
        personal_data = [] #would be good explore how to capture data at different points in time
        r = requests.get(base_url+"&limit=50", headers=headers)
        personal_data.append(json.loads(r.text))
        
        return personal_data
    
    def top_tracks(auth_token, time_range):
        
        base_url = 'https://api.spotify.com/v1/me/top/tracks?'
        #track_id = '6y0igZArWVi6Iz0rj35c1Y'

        #2. Authentication
        #3. Parameters -- would be stored with authentication
        headers = {
            "Authorization": f"Bearer {auth_token}"
        }

        #4. Create an empty list
        top_track_data = [] #would be good explore how to capture data at different points in time
        r = requests.get(base_url+f"time_range={time_range}"+"&&limit=50", headers=headers)
        top_track_data.append(json.loads(r.text))
        
        return top_track_data
        

        
class Data_Clean:
    
    def recently_played_clean(personal_data):
        track_ids = []
        album_ids = []
        artist_ids = []
        album_names = []
        artist_names = []
        track_names = []
        popularity_ls = []
        played_ats = []

        for i in range(len(personal_data[0]['items'])):
            track_ids.append(personal_data[0]['items'][i]['track']['id']) # Track ID
            album_ids.append(personal_data[0]['items'][i]['track']['album']['id']) # Albumn ID
            artist_ids.append(personal_data[0]['items'][i]['track']['artists'][0]['id']) # Artist ID
            album_names.append(personal_data[0]['items'][i]['track']['album']['name']) # Album Name
            artist_names.append(personal_data[0]['items'][i]['track']['artists'][0]['name']) # Artist Name
            track_names.append(personal_data[0]['items'][i]['track']['name']) # Track Name
            popularity_ls.append(personal_data[0]['items'][i]['track']['popularity']) # Track Popularity
            played_ats.append(personal_data[0]['items'][i]['played_at'])

        list_dic={'track_id':track_ids,
              'album_id':album_ids,
              'artist_id':artist_ids,
              'track_name':track_names,
              'artist_name':artist_names,
              'album_name':album_names,
              'track_popularity':popularity_ls,
              'time_palyed': played_ats
        }

        df1=pd.DataFrame(list_dic)

        # Create empty column to append data to
        df1['local_time'] = ''

        for i in range(len(personal_data[0]['items'])):
            # Convert UTC to local time zone
            utc_time = dateutil.parser.parse(df1['time_palyed'].iloc[i]).astimezone(get_localzone())
            # Format date/time
            local_time= utc_time.strftime('%Y-%m-%d %H:%M:%S')
            local_time

            df1['local_time'].iloc[i] = local_time

        return df1
    
    def top_tracks_clean(top_track_data):
        track_idss = []
        album_idss = []
        artist_idss = []
        album_namess = []
        album_relase_datess = []
        artist_namess = []
        popularity_lss = []
        track_namess = []

        for i in range(len(top_track_data[0]['items'])):
            track_idss.append(top_track_data[0]['items'][i]['id']) # Track ID
            album_idss.append(top_track_data[0]['items'][i]['album']['id']) # Album ID
            artist_idss.append(top_track_data[0]['items'][i]['album']['artists'][0]['id'])
            album_namess.append(top_track_data[0]['items'][i]['album']['name']) # Album Name
            album_relase_datess.append(top_track_data[0]['items'][i]['album']['release_date'])
            artist_namess.append(top_track_data[0]['items'][i]['album']['artists'][0]['name']) # Artist Name
            popularity_lss.append(top_track_data[0]['items'][i]['popularity'])
            track_namess.append(top_track_data[0]['items'][i]['name']) # Track Name
            
        list_dic2 = {'track_id':track_idss,
           'album_id':album_idss,
           'artist_id':artist_idss,
           'track_name':track_namess,
           'album_name':album_namess,
           'artist_name':artist_namess,
           'track_popularity':popularity_lss,
           'album_relase_date':album_relase_datess,
                  }
        
        df2=pd.DataFrame(list_dic2)
        
        return df2


In [65]:
token = Spotify_Auth.auth(Spotify_Auth.client_id, Spotify_Auth.client_secret, Spotify_Auth.username)
token

'BQBgeYBDmQHopWHUk95n23UVCMabbHSv1nixywgsa9o-hEHtTgwjBIbHXJExahgGJtkDJ8nqam_MdXtofOH7qAPAuO3pj09XUaYIlxTlHqLDByMx5qevvLIoGqmgA4FdK6PYFt8d0hlZco9SgAX9urI5JoXNITdhtJvW5U5rK9f_sTQllLcZHb_s8gsuJm_cE82-BeXzURe8uERCpMLCeNrkhmv6dcokI6rZ'

In [28]:
Data_Clean.recently_played_clean(Spotify_Data.recently_played(token))

,track_id,album_id,artist_id,track_name,artist_name,album_name,track_popularity,time_palyed,local_time
0,4FE9SgxcOoCNuGnaaU8TXz,2oiJM8vFGpxrtGtFfJWhJv,6ltzsmQQbmdoHHbLZ4ZN25,When the Night is Over,Lord Huron,Vide Noir,63,2020-11-22T23:59:31.165Z,2020-11-22 17:59:31
1,6CCjbWdVB3BvHcSbjRaHwS,2oiJM8vFGpxrtGtFfJWhJv,6ltzsmQQbmdoHHbLZ4ZN25,The Balancer's Eye,Lord Huron,Vide Noir,42,2020-11-22T23:54:18.942Z,2020-11-22 17:54:18
2,5exRDB26w6vi10fagGtgJS,2oiJM8vFGpxrtGtFfJWhJv,6ltzsmQQbmdoHHbLZ4ZN25,Back from the Edge,Lord Huron,Vide Noir,43,2020-11-22T23:49:48.504Z,2020-11-22 17:49:48
3,7bUgWgmd1yTYu351xFBLDL,2oiJM8vFGpxrtGtFfJWhJv,6ltzsmQQbmdoHHbLZ4ZN25,Secret of Life,Lord Huron,Vide Noir,44,2020-11-22T23:46:43.678Z,2020-11-22 17:46:43
4,2g9LXMNIBCqmngHpH9nXv0,2oiJM8vFGpxrtGtFfJWhJv,6ltzsmQQbmdoHHbLZ4ZN25,Wait by the River,Lord Huron,Vide Noir,59,2020-11-22T23:42:52.696Z,2020-11-22 17:42:52
5,6m4p3NyVyVPTkyPj59UhZH,2oiJM8vFGpxrtGtFfJWhJv,6ltzsmQQbmdoHHbLZ4ZN25,Ancient Names (Part II),Lord Huron,Vide Noir,42,2020-11-22T23:39:40.533Z,2020-11-22 17:39:40
6,6YrnbbmCTE5423w3qBdxqA,2oiJM8vFGpxrtGtFfJWhJv,6ltzsmQQbmdoHHbLZ4ZN25,Ancient Names (Part I),Lord Huron,Vide Noir,47,2020-11-22T23:37:33.669Z,2020-11-22 17:37:33
7,5UQKkiYuxg0oe21ACerQ7u,2oiJM8vFGpxrtGtFfJWhJv,6ltzsmQQbmdoHHbLZ4ZN25,Never Ever,Lord Huron,Vide Noir,46,2020-11-22T23:31:30.415Z,2020-11-22 17:31:30
8,6BqmgqzgKNxPszu1I6gOpn,2oiJM8vFGpxrtGtFfJWhJv,6ltzsmQQbmdoHHbLZ4ZN25,Lost in Time and Space,Lord Huron,Vide Noir,50,2020-11-22T23:28:36.804Z,2020-11-22 17:28:36
9,6BqmgqzgKNxPszu1I6gOpn,2oiJM8vFGpxrtGtFfJWhJv,6ltzsmQQbmdoHHbLZ4ZN25,Lost in Time and Space,Lord Huron,Vide Noir,50,2020-11-22T23:24:01.712Z,2020-11-22 17:24:01


In [71]:
Data_Clean.top_tracks_clean(Spotify_Data.top_tracks(token, Spotify_UserInput.user_timerange(Spotify_UserInput.user_time_range, Spotify_UserInput.time_range_reminder)))

How far back would you like to go?
Select time range by typing the number 1-3, your choices: 
1. short_term = approximately last 4 weeks
2. medium_term = approximately last 6 months
3. long_term = calculated from several years of data and including all new data as it becomes available
Your input: 
Time range #: 1
Pulling 'short_term' data...


,track_id,album_id,artist_id,track_name,album_name,artist_name,track_popularity,album_relase_date
0,3AVrVz5rK8Hrqo9YGiVGN5,5bP82ZIls6rzhpf5Qu6AzC,1QAJqy2dA3ihHBFIHRphZj,Apocalypse,Cigarettes After Sex,Cigarettes After Sex,70,2017-06-09
1,7LX4aXtvImhdgUHQBlo8Kp,5bP82ZIls6rzhpf5Qu6AzC,1QAJqy2dA3ihHBFIHRphZj,K.,Cigarettes After Sex,Cigarettes After Sex,61,2017-06-09
2,7IY2JF7AlVN5uvIUBVnSJd,10fSu9RLAJlasIup4ylHPM,2eam0iDomRHGBypaDQLwWI,chinatown (feat. Bruce Springsteen),chinatown (feat. Bruce Springsteen),Bleachers,63,2020-11-16
3,1gbgwb8AwRo9i42YaqWnj2,4Jkd7DR2cZjaVbQEQydXbX,2eam0iDomRHGBypaDQLwWI,45,45,Bleachers,54,2020-11-15
4,3GhsBdS9ulPK3KCdwHRPhG,6w9osZGOS8B7XWW51SBwFT,1QAJqy2dA3ihHBFIHRphZj,Nothing's Gonna Hurt You Baby,I.,Cigarettes After Sex,63,2012-07-21
5,4pvb0WLRcMtbPGmtejJJ6y,2fenSS68JI1h4Fo296JfGr,06HL4z0CvFAxyc27GXpf02,exile (feat. Bon Iver),folklore,Taylor Swift,83,2020-07-24
6,31q2AsxMNpxHNENkWKG1j0,6eIESGFbd6HVMqlO8e1mYT,4YZ5ECfbM2xSTSQTJGBbO5,Brother,Hesitant Alien,Gerard Way,56,2014-08-22
7,1Bh0UzthW8pKEnYg7v40Oa,6fUVptFdeOgcqDvCZzM2kC,1QAJqy2dA3ihHBFIHRphZj,Heavenly,Cry,Cigarettes After Sex,57,2019-10-25
8,3VSYkAf6f46i5ZkXOHORq1,5bP82ZIls6rzhpf5Qu6AzC,1QAJqy2dA3ihHBFIHRphZj,Sweet,Cigarettes After Sex,Cigarettes After Sex,58,2017-06-09
9,3e3d7FRtLL43hvnqUeJAwq,31i8FCqq0rEND2VDPzgkZY,67WNUxmM7y4WzHPAVzBu3E,Holiday-ish (feat. Dylan Minnette),Holiday-ish (feat. Dylan Minnette),The Regrettes,38,2019-10-25
